## First dataset

### Importing required libraries

In [ ]:
pip install python-sat

In [ ]:
pip install bnlearn

In [ ]:
pip install pysmt

In [ ]:
from pysat.solvers import Minisat22
from pysat.formula import CNF
import bnlearn as bn
from tabulate import tabulate

In [ ]:
import pandas as pd
from pysmt.shortcuts import Symbol, And, Not, Iff, Or, get_env
from pysmt.typing import BOOL
from pysmt.smtlib.parser import SmtLibParser
from pysat.formula import CNF
from pysat.solvers import Solver
from sympy import symbols
from sympy import symbols, Rational

# Importing dataset


* Asia: BN of eight binary variables



In [134]:
datasets = ["asia"]
dataset = datasets[0]
df = bn.import_DAG(dataset, CPD = True)
CPDs = bn.print_CPD(df, verbose = 1)

[bnlearn] >Import <asia>
[bnlearn] >Loading bif file </usr/local/lib/python3.10/dist-packages/datazets/data/asia.bif>
[bnlearn] >Check whether CPDs sum up to one.


Here is how the CPDs table looks like:

In [ ]:
CPDs

{'asia':    asia     p
 0     0  0.01
 1     1  0.99,
 'bronc':    bronc  smoke    p
 0      0      0  0.6
 1      0      1  0.3
 2      1      0  0.4
 3      1      1  0.7,
 'dysp':    dysp  bronc  either    p
 0     0      0       0  0.9
 1     0      0       1  0.8
 2     0      1       0  0.7
 3     0      1       1  0.1
 4     1      0       0  0.1
 5     1      0       1  0.2
 6     1      1       0  0.3
 7     1      1       1  0.9,
 'either':    either  lung  tub    p
 0       0     0    0  1.0
 1       0     0    1  1.0
 2       0     1    0  1.0
 3       0     1    1  0.0
 4       1     0    0  0.0
 5       1     0    1  0.0
 6       1     1    0  0.0
 7       1     1    1  1.0,
 'lung':    lung  smoke     p
 0     0      0  0.10
 1     0      1  0.01
 2     1      0  0.90
 3     1      1  0.99,
 'smoke':    smoke    p
 0      0  0.5
 1      1  0.5,
 'tub':    tub  asia     p
 0    0     0  0.05
 1    0     1  0.01
 2    1     0  0.95
 3    1     1  0.99,
 'xray':    xray  ei

### Utilities

1. Find the parent of every variable. This will help in introducing the right number of the variables for the child node, according to the combinations of the parent Truth assignments

In [ ]:
def get_parents_dict(dataframes_dict):
    parent_dict = {}

    for key, df in dataframes_dict.items():
        # Identify parent columns by excluding the target column and the probability column 'p'
        parent_columns = [col for col in df.columns if col not in {key, 'p'}]
        parent_dict[key] = parent_columns

    return parent_dict

In [ ]:
parent_dict = get_parents_dict(CPDs)

In [ ]:
parent_dict

{'asia': [],
 'bronc': ['smoke'],
 'dysp': ['bronc', 'either'],
 'either': ['lung', 'tub'],
 'lung': ['smoke'],
 'smoke': [],
 'tub': ['asia'],
 'xray': ['either']}

# Encoding BN in WMC

Step 1: Create new propositional variables

In [ ]:
def create_propositional_variables(cpt, parent_dict):

    """
    Create propositional variables for a given Conditional Probability Table (CPT) and its parent relationships.

    Parameters:
    - cpt (dict): A dictionary representing the Conditional Probability Table (CPT). The keys are variable names.
    - parent_dict (dict): A dictionary where keys are variable names and values are lists of parent variables.

    Returns:
    - all_vars (list): A list of all variable names extracted from the CPT.
    - var_symbols (dict): A dictionary mapping each variable name to its corresponding symbolic representation.
    - indicator_vars (dict): A dictionary containing indicator variables. For root nodes, the indicator variable is the
      same as the variable symbol. For non-root nodes, indicator variables are created for each possible configuration
      of parent variables.

    Example:
    >>> cpt = {'A': [0.2, 0.8], 'B': [0.5, 0.5]}
    >>> parent_dict = {'B': ['A']}
    >>> all_vars, var_symbols, indicator_vars = create_propositional_variables(cpt, parent_dict)
    >>> all_vars
    ['A', 'B']
    >>> var_symbols
    {'A': A, 'B': B}
    >>> indicator_vars
    {'A': A, 'B': B, 'B00': B00, 'B01': B01, 'B10': B10, 'B11': B11}
    """

    all_vars = list(cpt.keys())

    # Create symbols for all variables
    var_symbols = {var: symbols(var) for var in all_vars}

    # Create indicator variables
    indicator_vars = {}
    for var in all_vars:
        parents = parent_dict.get(var, [])
        if not parents:  # For root nodes
            indicator_vars[var] = symbols(var)
        else:

          indicator_vars[var] = symbols(var)

          from itertools import product
          num_parents = len(parents)
          for config in product([0, 1], repeat=num_parents):
            indicator_name = f"{var}{''.join(map(str, config))}"
            indicator_vars[indicator_name] = symbols(indicator_name)

    return all_vars, var_symbols, indicator_vars

In [ ]:
all_vars, var_symbols, indicator_vars = create_propositional_variables(CPDs, parent_dict)

Looking at the result: Despite the variables presented in the CPTs table, I have introduced new variables for each chiled node based on the number of parents. For example, the state variable ''dysp has two parents, thereforeI have introduced 4 new variables, each of them represents the combinations of different truth assignments of parents ['bronc', 'either']

In [ ]:
indicator_vars

{'asia': asia,
 'bronc': bronc,
 'bronc0': bronc0,
 'bronc1': bronc1,
 'dysp': dysp,
 'dysp00': dysp00,
 'dysp01': dysp01,
 'dysp10': dysp10,
 'dysp11': dysp11,
 'either': either,
 'either00': either00,
 'either01': either01,
 'either10': either10,
 'either11': either11,
 'lung': lung,
 'lung0': lung0,
 'lung1': lung1,
 'smoke': smoke,
 'tub': tub,
 'tub0': tub0,
 'tub1': tub1,
 'xray': xray,
 'xray0': xray0,
 'xray1': xray1}

Step 2, 3, 4: The function below assing the weights to the literals based on the following rules:

*   Assing the weight to the literal that has no parent as the
*   For the new introduced positive literal, given the truth values of the parents, check the entry in the table and assign it
*  For the negative ones, substruct the positive weight from one
*   For the other literals, that has no entry in th table, assign 1 as weight value


In [ ]:
def assign_weights(cpt, parent_dict, indicator_vars):

    """
    Assigns weights to indicator variables based on conditional probability tables (CPTs) and parent-child relationships.

    Parameters:
    - cpt (dict): A dictionary where the keys are variable names and the values are pandas DataFrames. Each DataFrame
      contains the CPT for the variable, with the last column being the probability 'p' and the other columns representing
      the variable and its parent values.
    - parent_dict (dict): A dictionary where the keys are variable names and the values are lists of parent variable names.
    - indicator_vars (dict): A dictionary mapping variable names and their configurations (as strings) to indicator variable
      symbols.

    Returns:
    - dict: A dictionary where the keys are indicator variable symbols and their negations (as sympy symbols) and the values
      are the assigned weights (float). The weights represent the probabilities for the positive literals and their complements
      for the negative literals.
    """
    weights = {}

    for var, df in cpt.items():
        parents = parent_dict.get(var, [])

        if not parents:  # Root node
            weight = df.loc[df[var] == 1, 'p'].values[0]
            weights[indicator_vars[var]] = round(float(weight),6)
        else:
            for _, row in df.iterrows():
                config = tuple(row[parents])
                prob = row['p']
                indicator_name = f"{var}{''.join(map(str, map(int, config)))}"
                weights[indicator_vars[indicator_name]] = round(float(prob),6)

    # Set negative weights
    neg_weights = {symbols(f"¬{var}"): round((1 - weight),6) for var, weight in weights.items()}

    # Combine positive and negative weights
    all_weights = {**weights, **neg_weights}

    # Set weights for all other literals to 1
    all_vars = set(indicator_vars.values())
    for var in all_vars:
        if var not in all_weights:
          neg_var = symbols(f"¬{var}")
          all_weights[var] = 1.0
          all_weights[neg_var] = 1.0

    return all_weights

In [ ]:
all_weights = assign_weights(CPDs, parent_dict, indicator_vars)

In [ ]:
all_weights

{asia: 0.99,
 bronc0: 0.4,
 bronc1: 0.7,
 dysp00: 0.1,
 dysp01: 0.2,
 dysp10: 0.3,
 dysp11: 0.9,
 either00: 0.0,
 either01: 0.0,
 either10: 0.0,
 either11: 1.0,
 lung0: 0.9,
 lung1: 0.99,
 smoke: 0.5,
 tub0: 0.95,
 tub1: 0.99,
 xray0: 0.02,
 xray1: 0.95,
 ¬asia: 0.01,
 ¬bronc0: 0.6,
 ¬bronc1: 0.3,
 ¬dysp00: 0.9,
 ¬dysp01: 0.8,
 ¬dysp10: 0.7,
 ¬dysp11: 0.1,
 ¬either00: 1.0,
 ¬either01: 1.0,
 ¬either10: 1.0,
 ¬either11: 0.0,
 ¬lung0: 0.1,
 ¬lung1: 0.01,
 ¬smoke: 0.5,
 ¬tub0: 0.05,
 ¬tub1: 0.01,
 ¬xray0: 0.98,
 ¬xray1: 0.05,
 tub: 1.0,
 ¬tub: 1.0,
 bronc: 1.0,
 ¬bronc: 1.0,
 either: 1.0,
 ¬either: 1.0,
 xray: 1.0,
 ¬xray: 1.0,
 lung: 1.0,
 ¬lung: 1.0,
 dysp: 1.0,
 ¬dysp: 1.0}

Step 5. This function create the implications to be used from Sat Solver.

In [ ]:
import itertools

def generate_equivalences(dataframes_dict):
    """
    Generates logical equivalences based on conditional probability tables (CPTs) provided in a dictionary of DataFrames.

    Parameters:
    - dataframes_dict (dict): A dictionary where the keys are target variable names and the values are pandas DataFrames.
      Each DataFrame contains the CPT for the target variable, with columns representing the parent variables, the target
      variable, and a probability column 'p'.

    Returns:
    - list: A list of strings, where each string is a logical equivalence derived from the CPTs.
    """
    equivalences = []

    for key, df in dataframes_dict.items():
        # Identify parent columns by excluding the target column and the probability column 'p'
        parent_columns = [col for col in df.columns if col not in {key, 'p'}]

        if not parent_columns:
            continue

        # Generate all binary combinations of parent columns
        k = len(parent_columns)
        for b in itertools.product([0, 1], repeat=k):
            pb = f"{key}{''.join(map(str, b))}"
            parents_true = [f"{parent}" for parent, bit in zip(parent_columns, b) if bit == 1]
            parents_false = [f"¬{parent}" for parent, bit in zip(parent_columns, b) if bit == 0]
            parent_clauses = " ∧ ".join(parents_true + parents_false)
            equivalence = f"{key} ∧ {parent_clauses} → {pb}"

            equivalences.append(equivalence)


            neg_pb = f"¬{pb}"
            neg_key = f"¬{key}"
            neg_parent_clauses = " ∧ ".join(parents_true + parents_false)
            neg_equivalence = f"{neg_key} ∧ {neg_parent_clauses} → {neg_pb}"


            equivalences.append(neg_equivalence)

    return equivalences


In [ ]:
equivalences = generate_equivalences(CPDs)

In [ ]:
equivalences

['bronc ∧ ¬smoke → bronc0',
 '¬bronc ∧ ¬smoke → ¬bronc0',
 'bronc ∧ smoke → bronc1',
 '¬bronc ∧ smoke → ¬bronc1',
 'dysp ∧ ¬bronc ∧ ¬either → dysp00',
 '¬dysp ∧ ¬bronc ∧ ¬either → ¬dysp00',
 'dysp ∧ either ∧ ¬bronc → dysp01',
 '¬dysp ∧ either ∧ ¬bronc → ¬dysp01',
 'dysp ∧ bronc ∧ ¬either → dysp10',
 '¬dysp ∧ bronc ∧ ¬either → ¬dysp10',
 'dysp ∧ bronc ∧ either → dysp11',
 '¬dysp ∧ bronc ∧ either → ¬dysp11',
 'either ∧ ¬lung ∧ ¬tub → either00',
 '¬either ∧ ¬lung ∧ ¬tub → ¬either00',
 'either ∧ tub ∧ ¬lung → either01',
 '¬either ∧ tub ∧ ¬lung → ¬either01',
 'either ∧ lung ∧ ¬tub → either10',
 '¬either ∧ lung ∧ ¬tub → ¬either10',
 'either ∧ lung ∧ tub → either11',
 '¬either ∧ lung ∧ tub → ¬either11',
 'lung ∧ ¬smoke → lung0',
 '¬lung ∧ ¬smoke → ¬lung0',
 'lung ∧ smoke → lung1',
 '¬lung ∧ smoke → ¬lung1',
 'tub ∧ ¬asia → tub0',
 '¬tub ∧ ¬asia → ¬tub0',
 'tub ∧ asia → tub1',
 '¬tub ∧ asia → ¬tub1',
 'xray ∧ ¬either → xray0',
 '¬xray ∧ ¬either → ¬xray0',
 'xray ∧ either → xray1',
 '¬xray ∧ ei

# Utilities to transform the problem for the SAT solver

1. Make sure that every positive literal is given a distinct positive integer and that every negative literal is given the same integer with a negative sign by mapping each literal to an integer value.

In [ ]:
def create_symbol_mapping(equivalences):
    symbol_mapping = {}
    counter = 1

    for implication in equivalences:
        literals = implication.replace('→', '∧').replace('¬', '').split('∧')
        for literal in literals:
            literal = literal.strip()
            if literal not in symbol_mapping:
                symbol_mapping[literal] = counter
                symbol_mapping[f'¬{literal}'] = -counter
                counter += 1

    return symbol_mapping

In [ ]:
symbol_mapping = create_symbol_mapping(equivalences)

In [ ]:
symbol_mapping

{'bronc': 1,
 '¬bronc': -1,
 'smoke': 2,
 '¬smoke': -2,
 'bronc0': 3,
 '¬bronc0': -3,
 'bronc1': 4,
 '¬bronc1': -4,
 'dysp': 5,
 '¬dysp': -5,
 'either': 6,
 '¬either': -6,
 'dysp00': 7,
 '¬dysp00': -7,
 'dysp01': 8,
 '¬dysp01': -8,
 'dysp10': 9,
 '¬dysp10': -9,
 'dysp11': 10,
 '¬dysp11': -10,
 'lung': 11,
 '¬lung': -11,
 'tub': 12,
 '¬tub': -12,
 'either00': 13,
 '¬either00': -13,
 'either01': 14,
 '¬either01': -14,
 'either10': 15,
 '¬either10': -15,
 'either11': 16,
 '¬either11': -16,
 'lung0': 17,
 '¬lung0': -17,
 'lung1': 18,
 '¬lung1': -18,
 'asia': 19,
 '¬asia': -19,
 'tub0': 20,
 '¬tub0': -20,
 'tub1': 21,
 '¬tub1': -21,
 'xray': 22,
 '¬xray': -22,
 'xray0': 23,
 '¬xray0': -23,
 'xray1': 24,
 '¬xray1': -24}

2. Use the same symbol mapping to update the literals in the generated equivalences

In [66]:
def update_equivalences(equivalences, symbol_mapping):
    updated_equivalences = []

    for implication in equivalences:
        updated_implication = implication

        sorted_items = sorted(symbol_mapping.items(), key=lambda x: len(x[0]), reverse=True)

        for literal, number in sorted_items:
            updated_implication = updated_implication.replace(literal, str(number))

        updated_equivalences.append(updated_implication)

    return updated_equivalences

In [67]:
updated_equivalences = update_equivalences(equivalences, symbol_mapping)

In [78]:
updated_equivalences

['1 ∧ -2 → 3',
 '-1 ∧ -2 → -3',
 '1 ∧ 2 → 4',
 '-1 ∧ 2 → -4',
 '5 ∧ -1 ∧ -6 → 7',
 '-5 ∧ -1 ∧ -6 → -7',
 '5 ∧ 6 ∧ -1 → 8',
 '-5 ∧ 6 ∧ -1 → -8',
 '5 ∧ 1 ∧ -6 → 9',
 '-5 ∧ 1 ∧ -6 → -9',
 '5 ∧ 1 ∧ 6 → 10',
 '-5 ∧ 1 ∧ 6 → -10',
 '6 ∧ -11 ∧ -12 → 13',
 '-6 ∧ -11 ∧ -12 → -13',
 '6 ∧ 12 ∧ -11 → 14',
 '-6 ∧ 12 ∧ -11 → -14',
 '6 ∧ 11 ∧ -12 → 15',
 '-6 ∧ 11 ∧ -12 → -15',
 '6 ∧ 11 ∧ 12 → 16',
 '-6 ∧ 11 ∧ 12 → -16',
 '11 ∧ -2 → 17',
 '-11 ∧ -2 → -17',
 '11 ∧ 2 → 18',
 '-11 ∧ 2 → -18',
 '12 ∧ -19 → 20',
 '-12 ∧ -19 → -20',
 '12 ∧ 19 → 21',
 '-12 ∧ 19 → -21',
 '22 ∧ -6 → 23',
 '-22 ∧ -6 → -23',
 '22 ∧ 6 → 24',
 '-22 ∧ 6 → -24']

3. In order to have the formula in the required format by the SAT solver, I have build two auxilary functions to be used by the final function that prepares the final formula for such a solver.



> Step 1: Transform the implication into CNF

After applying this function to an implication this is how it will look like below, which is not in the format required by MiniSat solver. I will use another function to parse the formula to the required format:

   

*   implication = "22 ∧ 6 → 24"
*   cnf = implication_to_cnf(implication)
*   print(cnf)
*   (¬22 ∨ ¬6 ∨ 24)



> Step 2: Format the resulted CNF

*   [¬22 , ¬6, 24]






   
   
   


In [100]:
from sympy import symbols, Or, Not, And
from sympy.logic.boolalg import to_cnf
def implication_to_cnf(implication):
    """
    Converts a logical implication into its Conjunctive Normal Form (CNF).

    Parameters:
    - implication (str): A string representing a logical implication in the form "LHS → RHS",
      where LHS is a conjunction of literals separated by '∧' and RHS is a single literal.
      Negated literals are prefixed with '¬'.

    Returns:
    - sympy.logic.boolalg.Boolean: A SymPy expression representing the CNF of the given implication.

    The function performs the following steps:
    1. Splits the implication into its Left-Hand Side (LHS) and Right-Hand Side (RHS).
    2. Converts the literals in the LHS and RHS to SymPy symbols, handling negations appropriately.
    3. Creates a SymPy expression for the implication.
    4. Converts the implication expression to its CNF using SymPy's `to_cnf` function.
    """
    # Split implication into LHS and RHS
    lhs, rhs = implication.split('→')
    lhs = lhs.strip()
    rhs = rhs.strip()

    # Convert to sympy symbols
    lhs_symbols = [symbols(lit.strip()) if not lit.strip().startswith('¬') else Not(symbols(lit.strip()[1:])) for lit in lhs.split('∧')]
    rhs_symbol = symbols(rhs) if not rhs.startswith('¬') else Not(symbols(rhs[1:].strip()))

    # Create implication and convert to CNF
    implication_expr = Or(Not(And(*lhs_symbols)), rhs_symbol)
    cnf_expr = to_cnf(implication_expr, simplify=True)

    return cnf_expr

In [101]:
from sympy import Or, Not, Symbol

def format_cnf_clause(clause):

    """
    Formats a CNF clause into a list of integers.

    Parameters:
    - clause (sympy.logic.boolalg.Boolean): A SymPy Boolean expression representing a CNF clause.
      The clause can be an instance of `Or`, `Not`, or `Symbol`.

    Returns:
    - list: A list of integers where each integer corresponds to a literal in the clause.
      Positive integers represent positive literals, and negative integers represent negated literals.

    The function performs the following steps:
    1. Ensures the clause is an `Or` expression. If it is not, it converts it to an `Or` expression.
    2. Iterates over the arguments of the `Or` expression.
    3. For each argument, if it is a negation (`Not`), it extracts the literal and negates its integer value.
       If it is a positive literal (`Symbol`), it converts it directly to an integer.
    4. Collects the formatted literals in a list and returns it.
    """
    if not isinstance(clause, Or):
        clause = Or(clause)

    formatted_clause = []

    for arg in clause.args:
        if isinstance(arg, Not):
            literal = -int(str(arg.args[0]))
        elif isinstance(arg, Symbol):
            literal = int(str(arg))
        else:
            literal = int(str(arg))
        formatted_clause.append(literal)

    return formatted_clause

In [ ]:
"""
def format_cnf_clause2(clause_str):

    literals = clause_str.split('|')
    formatted_clause = []

    for literal in literals:
        literal = literal.strip()
        if literal.startswith('~-'):
            formatted_clause.append(int(literal[2:]))  # Remove '~-' and convert to positive integer
        elif literal.startswith('~'):
            formatted_clause.append(-int(literal[1:]))  # Remove '~' and negate
        else:
            formatted_clause.append(int(literal))

    return formatted_clause

"""

4. Finally, we can use the functions implemented to generate the CNF clauses to be inserted in SAT solver

In [96]:
def transform_to_cnf2(updated_equivalences):

  cnf = CNF()
  for implication in updated_equivalences:

    cnf_expr = implication_to_cnf(implication)
    formated_one = format_cnf_clause(cnf_expr)
    cnf.append(formated_one)

  return cnf


In [102]:
cnf  = transform_to_cnf2(updated_equivalences)

In [103]:
cnf

CNF(from_string='p cnf 24 32\n3 2 -1 0\n-3 1 2 0\n4 -1 -2 0\n-4 1 -2 0\n7 1 6 -5 0\n-7 1 5 6 0\n8 1 -5 -6 0\n-8 1 5 -6 0\n9 6 -1 -5 0\n-9 5 6 -1 0\n10 -1 -5 -6 0\n-10 5 -1 -6 0\n13 11 12 -6 0\n-13 11 12 6 0\n14 11 -12 -6 0\n-14 11 6 -12 0\n15 12 -11 -6 0\n-15 12 6 -11 0\n16 -11 -12 -6 0\n-16 6 -11 -12 0\n17 2 -11 0\n-17 11 2 0\n18 -11 -2 0\n-18 11 -2 0\n20 19 -12 0\n-20 12 19 0\n21 -12 -19 0\n-21 12 -19 0\n23 6 -22 0\n-23 22 6 0\n24 -22 -6 0\n-24 22 -6 0')

# Utilities to prepare weights for our model

1. Recall that the dictionary that contains the weights for all the literals already exist. Now, I need to mapp each of the literals to its integer value based on the symbol_mapping dictionary

In [108]:
def apply_symbol_mapping_to_weights(weight_dict, symbol_mapping):

    """
    Applies a symbol mapping to a dictionary of weights, converting symbol keys to their corresponding mapped integer values.

    Parameters:
    - weight_dict (dict): A dictionary where keys are symbol strings and values are their corresponding weights (floats).
    - symbol_mapping (dict): A dictionary that maps symbol strings to integer values. Positive and negative literals
                             should have the same integer value but opposite signs.

    Returns:
    - dict: A new dictionary where the keys are the mapped integer values from `symbol_mapping` and the values are the
            corresponding weights from `weight_dict`.
    """

    mapped_weight_dict = {}


    for key, value in weight_dict.items():
      if f'{key}' in symbol_mapping:
        mapped_key = symbol_mapping[f'{key}']
        mapped_weight_dict[mapped_key] = value

    return mapped_weight_dict

In [115]:
mapped_weight_dict = apply_symbol_mapping_to_weights(all_weights, symbol_mapping)

2. Based on our PHI, the SAT solver returns the models that satisfy it. Essentially, we need to get the weights from our dictionary of the literals that compose this model

In [110]:
def extract_weights_of_literals_for_given_model(mapped_weight_dict, model):

  """
    Extracts the weights of literals for a given model based on a mapped weight dictionary.

    Parameters:
    - mapped_weight_dict (dict): A dictionary where keys are integer representations of literals and values are their corresponding weights (floats).
    - model (list of int): A list of integers representing the literals in the given model.

    Returns:
    - list of float: A list of weights corresponding to the literals in the given model.
    """


  literal_weight_values = []


  for key in model:

    literal_weight_values.append(mapped_weight_dict[key])

  return literal_weight_values


3. As the literals that compose this model are in a conjuction, we need to multiply their weights. When calculatin the weight of our PHI, as we will see, we need to sum all of them

In [112]:
def weight_of_given_model(literal_weight_values):

  product = 1
  for literal_weight in literal_weight_values:
    product *= literal_weight

  return product

4. Lastly, to make it easy to find the correspondent int mapping for a given literal when calculating the probablity, I use the function below

In [113]:
def extract_int_mapping(literal,symbol_mapping):

  representative_value = []

  for l in literal:
    r  = symbol_mapping[f'{l}']
    representative_value.append(r)

  return representative_value

# Test the functionalities created

1. Check if the WMC of PHI is 1

In [114]:
WMC_phi = 0
phi_b = Minisat22(bootstrap_with = cnf.clauses)
for models in phi_b.enum_models():
  WMC_phi += weight_of_given_model(extract_weights_of_literals_for_given_model(mapped_weight_dict, models))

print(round(WMC_phi,2))

1.0


2. Before each test, clean CNF clauses and initialize it with the initial clauses



> First experiment



In [122]:
# negation symbol   ¬
literal = ['asia','bronc','¬dysp','either','lung','smoke']

representative_value = extract_int_mapping(literal,symbol_mapping)

representative_value

[19, 1, -5, 6, 11, 2]

In [116]:
cnf = transform_to_cnf2(updated_equivalences)

In [117]:
# this is how I add the evidence
cnf.append([19])
cnf.append([1])
cnf.append([-5])
cnf.append([6])
cnf.append([11])
s1 = Minisat22(bootstrap_with=cnf.clauses)

WMC_den = 0
for models in s1.enum_models():
    WMC_den += weight_of_given_model(extract_weights_of_literals_for_given_model(mapped_weight_dict, models))

In [118]:
# this is how I add the query
cnf.append([-2])
s2 = Minisat22(bootstrap_with=cnf.clauses)

WMC_num = 0
for models in s2.enum_models():
    WMC_num += weight_of_given_model(extract_weights_of_literals_for_given_model(mapped_weight_dict, models))

In [119]:
print(round(WMC_num / WMC_den, 3))

0.342


In [120]:

model = bn.import_DAG('asia')
query = bn.inference.fit(model, variables=['smoke'], evidence={"asia":1, "bronc":1, "dysp":0, "either":1, 'lung':1}, joint = True)
print(query)

[bnlearn] >Import <asia>
[bnlearn] >Loading bif file </usr/local/lib/python3.10/dist-packages/datazets/data/asia.bif>
[bnlearn] >Check whether CPDs sum up to one.
[bnlearn] >Variable Elimination.
[bnlearn] >Warning: variable(s) [None] does not exists in DAG.
[bnlearn] >Data is stored in [query.df]
+----+---------+---------+
|    |   smoke |       p |
+====+=========+=========+
|  0 |       0 | 0.34188 |
+----+---------+---------+
|  1 |       1 | 0.65812 |
+----+---------+---------+
+----------+--------------+
| smoke    |   phi(smoke) |
+==========+==============+
| smoke(0) |       0.3419 |
+----------+--------------+
| smoke(1) |       0.6581 |
+----------+--------------+




> Second experiment



In [123]:
# negation symbol   ¬
literal = ['lung','bronc','¬dysp','xray','tub']

representative_value = extract_int_mapping(literal,symbol_mapping)

representative_value

[11, 1, -5, 22, 12]

In [129]:
cnf = transform_to_cnf2(updated_equivalences)

In [130]:
# this is how I add the evidence
cnf.append([11])
cnf.append([1])
cnf.append([-5])
cnf.append([22])
s1 = Minisat22(bootstrap_with=cnf.clauses)

WMC_den = 0
for models in s1.enum_models():
    WMC_den += weight_of_given_model(extract_weights_of_literals_for_given_model(mapped_weight_dict, models))

In [131]:
# this is how I add the query
cnf.append([-12])
s2 = Minisat22(bootstrap_with=cnf.clauses)

WMC_num = 0
for models in s2.enum_models():
    WMC_num += weight_of_given_model(extract_weights_of_literals_for_given_model(mapped_weight_dict, models))



> Numerator contains clauses of PHI, question, and evidence


> Denominator contains clauses of PHI and evidence





In [132]:
print(round(WMC_num / WMC_den, 3))

0.002


In [128]:
model = bn.import_DAG('asia')
query = bn.inference.fit(model, variables=['tub'], evidence={'lung':1,'bronc':1,'dysp':0,'xray':1 }, joint = True)
print(query)

[bnlearn] >Import <asia>
[bnlearn] >Loading bif file </usr/local/lib/python3.10/dist-packages/datazets/data/asia.bif>
[bnlearn] >Check whether CPDs sum up to one.
[bnlearn] >Variable Elimination.
[bnlearn] >Warning: variable(s) [None] does not exists in DAG.
[bnlearn] >Data is stored in [query.df]
+----+-------+------------+
|    |   tub |          p |
+====+=======+============+
|  0 |     0 | 0.00154634 |
+----+-------+------------+
|  1 |     1 | 0.998454   |
+----+-------+------------+
+--------+------------+
| tub    |   phi(tub) |
+========+============+
| tub(0) |     0.0015 |
+--------+------------+
| tub(1) |     0.9985 |
+--------+------------+


# Second dataset

In [133]:
datasets = ["sprinkler"]

dataset = datasets[0]

df = bn.import_DAG(dataset, CPD = True)
CPDs_sprink = bn.print_CPD(df, verbose = 1)
print(CPDs_sprink.keys())

[bnlearn] >Import <sprinkler>
[bnlearn] >Check whether CPDs sum up to one.
dict_keys(['Cloudy', 'Sprinkler', 'Rain', 'Wet_Grass'])




1.   Print CPTs to see the structure
2.   Get parent dictionary
3.   Create propositional variables
4.   Assign weights to literals
5.   Generate implications
6.   Create literal mapping to int values
7.   Update implications with int values of literals
8.   Transform implications into CNF
9.   Update wight dictionary with int values of literals





In [135]:
CPDs_sprink

{'Cloudy':    Cloudy    p
 0       0  0.5
 1       1  0.5,
 'Sprinkler':    Sprinkler  Cloudy    p
 0          0       0  0.5
 1          0       1  0.9
 2          1       0  0.5
 3          1       1  0.1,
 'Rain':    Rain  Cloudy    p
 0     0       0  0.8
 1     0       1  0.2
 2     1       0  0.2
 3     1       1  0.8,
 'Wet_Grass':    Wet_Grass  Sprinkler  Rain     p
 0          0          0     0  1.00
 1          0          0     1  0.10
 2          0          1     0  0.10
 3          0          1     1  0.01
 4          1          0     0  0.00
 5          1          0     1  0.90
 6          1          1     0  0.90
 7          1          1     1  0.99}

In [136]:
parent_dict_sprink = get_parents_dict(CPDs_sprink)
all_vars_sprink, var_symbols_sprink, indicator_vars_sprink = create_propositional_variables(CPDs_sprink, parent_dict_sprink)
indicator_vars_sprink

{'Cloudy': Cloudy,
 'Sprinkler': Sprinkler,
 'Sprinkler0': Sprinkler0,
 'Sprinkler1': Sprinkler1,
 'Rain': Rain,
 'Rain0': Rain0,
 'Rain1': Rain1,
 'Wet_Grass': Wet_Grass,
 'Wet_Grass00': Wet_Grass00,
 'Wet_Grass01': Wet_Grass01,
 'Wet_Grass10': Wet_Grass10,
 'Wet_Grass11': Wet_Grass11}

In [137]:
all_weights_sprink = assign_weights(CPDs_sprink, parent_dict_sprink, indicator_vars_sprink)
all_weights_sprink

{Cloudy: 0.5,
 Sprinkler0: 0.5,
 Sprinkler1: 0.1,
 Rain0: 0.2,
 Rain1: 0.8,
 Wet_Grass00: 0.0,
 Wet_Grass01: 0.9,
 Wet_Grass10: 0.9,
 Wet_Grass11: 0.99,
 ¬Cloudy: 0.5,
 ¬Sprinkler0: 0.5,
 ¬Sprinkler1: 0.9,
 ¬Rain0: 0.8,
 ¬Rain1: 0.2,
 ¬Wet_Grass00: 1.0,
 ¬Wet_Grass01: 0.1,
 ¬Wet_Grass10: 0.1,
 ¬Wet_Grass11: 0.01,
 Rain: 1.0,
 ¬Rain: 1.0,
 Sprinkler: 1.0,
 ¬Sprinkler: 1.0,
 Wet_Grass: 1.0,
 ¬Wet_Grass: 1.0}

In [138]:
equivalences_sprink = generate_equivalences(CPDs_sprink)
equivalences_sprink

['Sprinkler ∧ ¬Cloudy → Sprinkler0',
 '¬Sprinkler ∧ ¬Cloudy → ¬Sprinkler0',
 'Sprinkler ∧ Cloudy → Sprinkler1',
 '¬Sprinkler ∧ Cloudy → ¬Sprinkler1',
 'Rain ∧ ¬Cloudy → Rain0',
 '¬Rain ∧ ¬Cloudy → ¬Rain0',
 'Rain ∧ Cloudy → Rain1',
 '¬Rain ∧ Cloudy → ¬Rain1',
 'Wet_Grass ∧ ¬Sprinkler ∧ ¬Rain → Wet_Grass00',
 '¬Wet_Grass ∧ ¬Sprinkler ∧ ¬Rain → ¬Wet_Grass00',
 'Wet_Grass ∧ Rain ∧ ¬Sprinkler → Wet_Grass01',
 '¬Wet_Grass ∧ Rain ∧ ¬Sprinkler → ¬Wet_Grass01',
 'Wet_Grass ∧ Sprinkler ∧ ¬Rain → Wet_Grass10',
 '¬Wet_Grass ∧ Sprinkler ∧ ¬Rain → ¬Wet_Grass10',
 'Wet_Grass ∧ Sprinkler ∧ Rain → Wet_Grass11',
 '¬Wet_Grass ∧ Sprinkler ∧ Rain → ¬Wet_Grass11']

In [139]:
symbol_mapping_sprink = create_symbol_mapping(equivalences_sprink)

In [140]:
updated_equivalences_sprink = update_equivalences(equivalences_sprink , symbol_mapping_sprink)
updated_equivalences_sprink

['1 ∧ -2 → 3',
 '-1 ∧ -2 → -3',
 '1 ∧ 2 → 4',
 '-1 ∧ 2 → -4',
 '5 ∧ -2 → 6',
 '-5 ∧ -2 → -6',
 '5 ∧ 2 → 7',
 '-5 ∧ 2 → -7',
 '8 ∧ -1 ∧ -5 → 9',
 '-8 ∧ -1 ∧ -5 → -9',
 '8 ∧ 5 ∧ -1 → 10',
 '-8 ∧ 5 ∧ -1 → -10',
 '8 ∧ 1 ∧ -5 → 11',
 '-8 ∧ 1 ∧ -5 → -11',
 '8 ∧ 1 ∧ 5 → 12',
 '-8 ∧ 1 ∧ 5 → -12']

In [141]:
cnf_sprink = transform_to_cnf2(updated_equivalences_sprink)
cnf_sprink

CNF(from_string='p cnf 12 16\n3 2 -1 0\n-3 1 2 0\n4 -1 -2 0\n-4 1 -2 0\n6 2 -5 0\n-6 2 5 0\n7 -2 -5 0\n-7 5 -2 0\n9 1 5 -8 0\n-9 1 5 8 0\n10 1 -5 -8 0\n-10 1 8 -5 0\n11 5 -1 -8 0\n-11 5 8 -1 0\n12 -1 -5 -8 0\n-12 8 -1 -5 0')

In [142]:
mapped_weight_dict_sprink = apply_symbol_mapping_to_weights(all_weights_sprink, symbol_mapping_sprink)

# Test functionalities in the second dataset

In [143]:
WMC_phi_sprink = 0
phi_b_sprink = Minisat22(bootstrap_with = cnf_sprink.clauses)
for models in phi_b_sprink.enum_models():
  WMC_phi_sprink += weight_of_given_model(extract_weights_of_literals_for_given_model(mapped_weight_dict_sprink, models))

print(round(WMC_phi_sprink,2))

1.0


In [163]:
# negation symbol   ¬
literal_sprink = ['Cloudy','Rain','¬Wet_Grass']

representative_value_sprink = extract_int_mapping(literal_sprink,symbol_mapping_sprink)

representative_value_sprink

[2, 5, -8]

In [164]:
cnf_sprink = transform_to_cnf2(updated_equivalences_sprink)

In [165]:
# this is how I add the evidence
cnf_sprink.append([2])
cnf_sprink.append([5])
s1_sprink = Minisat22(bootstrap_with=cnf_sprink.clauses)
WMC_den_sprink = 0
for models in s1_sprink.enum_models():
    WMC_den_sprink += weight_of_given_model(extract_weights_of_literals_for_given_model(mapped_weight_dict_sprink, models))

In [166]:
# this is how I add the query
cnf_sprink.append([-8])
s2_sprink = Minisat22(bootstrap_with=cnf_sprink.clauses)

WMC_num_sprink = 0
for models in s2_sprink.enum_models():
    WMC_num_sprink += weight_of_given_model(extract_weights_of_literals_for_given_model(mapped_weight_dict_sprink, models))

In [167]:
print(round(WMC_num_sprink / WMC_den_sprink, 3))

0.091


In [168]:
model = bn.import_DAG('sprinkler')
query = bn.inference.fit(model, variables=['Wet_Grass'], evidence={"Cloudy":1, 'Rain':1}, joint = True)
print(query)

[bnlearn] >Import <sprinkler>
[bnlearn] >Check whether CPDs sum up to one.
[bnlearn] >Variable Elimination.
[bnlearn] >Warning: variable(s) [None] does not exists in DAG.
[bnlearn] >Data is stored in [query.df]
+----+-------------+-------+
|    |   Wet_Grass |     p |
+====+=============+=======+
|  0 |           0 | 0.091 |
+----+-------------+-------+
|  1 |           1 | 0.909 |
+----+-------------+-------+
+--------------+------------------+
| Wet_Grass    |   phi(Wet_Grass) |
+==============+==================+
| Wet_Grass(0) |           0.0910 |
+--------------+------------------+
| Wet_Grass(1) |           0.9090 |
+--------------+------------------+
